# ACS AWS Training: Day 5
## Nathan Miles
06/15/2020

## Outline
- Review of last time
- Docker 101
    - Containers
- Example
    - Create a lambda funciton that will,
        - run SExtractor on an image to identify all sources,
        - generate a catalog of photometry for all sources found in the image
    - Containerize the Lambda function and its dependencies using `EC2`
        - `astropy`
        - `matplotlib`
        - `numpy`
        - `scipy`
        - `sep` (SExtractor in python)
    - Uploading the container to S3
    - Building a Lambda function using the container
    - Invoking the lambda function with `boto3`


## Docker
- Platform for containerizing applications
- Containers are standardized units of software
    - Packages up code and all its dependencies so it runs quickly and reliably in a variety computing environments
        - code, runtime, system tools, system libraries and settings
    - OS-level virtualization

<center><figure><img src="visual_representation_of_a_container.png" width="50%" height="50%" /><figcaption>Credit: <a href="https://www.docker.com/resources/what-container">What is a Container?</a></figcaption></figure></center>

## Example: Overview
- To circumvent any issues associated with installing/configuring Docker on your computer, we will build our container on a small EC2 Instance.
- An outline of the process is as follows,
    - Build the container on EC2
    - Upload the container to S3
    - Create a Lambda function and specify the container in S3 as the source code
    - Create a test event for the Lambda function



## Example: Download the Software
#### Place the files in the following directory: `~/acs_aws_day5_code`

- <a href="./day5_code/build.sh">build.sh</a>
    - Build script used by docker to create our container
- <a href="./day5_code/requirements.txt">requirements.txt</a>
    - `pip` requirements file specifying the packages to install
- <a href="./day5_code/extract_photometry.py">extract_photometry.py</a>
    - Python class for performing aperture photometry using Python wrapped <a href="https://sep.readthedocs.io/en/v1.0.x/">Source Extractor</a>
- <a href="./day5_code/datavis.py">datavis.py</a>
    - Convenience class for handling all data visualization 
- <a href="./day5_code/aws_source_extraction.py">aws_source_extraction.py</a>
    - Python module for running source extraction on AWS
    - This is the file that contains the `handler` for the Lambda function
- <a href="./day5_code/launch_instance.py">launch_instance.py</a>
    - Launch an EC2 instance using a specified launch template and key pair





## Containerizing the Lambda Function
- Launch an EC2 template to use for creating the Docker container

``` console
python launch_instance.py -template_id <TEMPLATE_ID> -keyname <KEYNAME>
```

- Copy over the `acs_aws_day5_code` directory to your EC2 instance

```console
scp -ri "path/to/key.pem" ~/acs_aws_day5_code ec2-user@public_dns
```

- Install and configure docker,
    - install: `sudo yum install docker`
    - start docker: `sudo service docker start`
    - set permissions: `sudo usermod -a -G docker ec2-user`


## Containerizing the Lambda Function Continued
- Log out of the EC2 instance
- Log back in and execute the command: `docker info`
    - If no output is shown, your docker installation failed
- Change directory to `~/acs_aws_day5_code`
- Execute the following commands to build your docker image
    - `docker pull amazonlinux:2018.03`
    - `docker run -v $(pwd):/outputs -it amazonlinux:2018.03 /bin/bash /outputs/build.sh`
        - The final container is `venv.zip`
        